# Global Mean EOF Analysis
Liam Kirkpatrick
ATM 552, Assignment 5
February 4, 2024

## Introduction

This code is modeled after Dennis Hartman's script on github (https://github.com/dennislhartmann/Objective_Analysis/tree/main).

## Import Packages

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import netcdf
from scipy.io import loadmat
from scipy.interpolate import griddata
import scipy.signal as sig
import xarray as xr
from cartopy import config
import cartopy.crs as ccrs
import pandas as pd
import netCDF4 as nc
from cartopy import config
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

# land mask package
from global_land_mask import globe

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## User Inputs

This should be the only place we need to change things for the rest of the script

In [2]:
if False:
    #  Global domain
    lat1=88.
    lat2=-88.
    lon1=0.
    lon2=360.

if True:
    # Traditional PDO region
    lat1=65.
    lat2=20.
    lon1=120.
    lon2=260.

# set years to begin and end for climatology
yrbeg = 1940
yrend = 2023

#set the years for eof analysis
yra = yrbeg
yrb = yrend

# central longitude for Robinson maps
cent_lon = -160.   

# behind the scenes bookkeeping on year selection
ds_yrmin = 1940 # min year of dataset
iyr1 = (yrbeg-ds_yrmin)*12
iyr2 = (yrend - ds_yrmin + 1)*12

# choices on removal
rem_globmean = True  # removes global mean anomaly from each grid cell
rem_ann = True    # removes annual cycle from each grid cell


In [3]:
## Import Data

# open using xarray
ds_disk = xr.open_dataset('../data/adaptor.mars.internal-1708137618.9186294-22048-4-1f8f03ca-bdfd-48d7-93b6-88f79b1175d3.grib',engine='cfgrib',) 
sst = ds_disk.sst
mslp = ds_disk.msl
lon = ds_disk.longitude
lat = ds_disk.latitude

# uncomment next line to see data format.
#ds_disk

Can't read index file '../data/adaptor.mars.internal-1708137618.9186294-22048-4-1f8f03ca-bdfd-48d7-93b6-88f79b1175d3.grib.923a8.idx'
Traceback (most recent call last):
  File "/Users/Liam/anaconda3/envs/atm552/lib/python3.11/site-packages/cfgrib/messages.py", line 545, in from_indexpath_or_filestream
    filestream_mtime = os.path.getmtime(filestream.path)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen genericpath>", line 55, in getmtime
FileNotFoundError: [Errno 2] No such file or directory: '../data/adaptor.mars.internal-1708137618.9186294-22048-4-1f8f03ca-bdfd-48d7-93b6-88f79b1175d3.grib'


FileNotFoundError: [Errno 2] No such file or directory: '../data/adaptor.mars.internal-1708137618.9186294-22048-4-1f8f03ca-bdfd-48d7-93b6-88f79b1175d3.grib'

## Clean up Data

In [ ]:
# extend longitude array
lonp = np.empty(len(lon)+1)
lonp[0:len(lon)] = lon
lonp[len(lon)]= lon[len(lon)-1]+lon[1]-lon[0]

# Calculate SST and MSLP Mean
sstm = np.nanmean(sst,0)
mslpm = np.nanmean(mslp,0)

# Here we are adding cyclic continuity to SST, so it plots without gaps
sstmp = np.empty([len(lat),len(lon)+1])
sstmp[:,0:len(lon)]= sstm
sstmp[:,len(lon+1)]= sstm[:,0]

# Here we are adding cyclic continuity to MSLP, so it plots without gaps
mslpmp = np.empty([len(lat),len(lon)+1])
mslpmp[:,0:len(lon)]= mslpm
mslpmp[:,len(lon+1)]= mslpm[:,0]

# clip out years I want to consider
sst1 = sst[iyr1:iyr2,:,:]
mslp1 = mslp[iyr1:iyr2,:,:]
print('sst clipped shape ',np.shape(sst))
print('mslp clipped shape ',np.shape(mslp))

## Make Grand Mean SST Plot

In [ ]:
col_mapc = 'RdYlBu_r'
col_mapr = 'RdYlBu_r'

# make figure - SST
plt.figure(figsize=(12, 4), dpi=100)

# set up axis. Use cent_lon set above
ax = plt.axes(projection=ccrs.Robinson(central_longitude=cent_lon))
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1.5, color='gray', alpha=0.5, linestyle='-')

# add color for sst
plt.contourf(lonp, lat, sstmp, 60, cmap=col_mapc,
             transform=ccrs.PlateCarree())

# Housekeeping
plt.title('Grand Mean SST')
ax.coastlines()
plt.grid()
plt.colorbar()
plt.show()



## Make grand mean MSLP

In [ ]:
# make figure - MSLP
plt.figure(figsize=(12, 4), dpi=100)

# set up axis. Use cent_lon set above
ax = plt.axes(projection=ccrs.Robinson(central_longitude=cent_lon))
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=1.5, color='gray', alpha=0.5, linestyle='-')

# add color for mslp
plt.contourf(lonp, lat, mslpmp, 60, cmap=col_mapc,
             transform=ccrs.PlateCarree())

# Housekeeping
plt.title('Grand Mean MSLP')
ax.coastlines()
plt.grid()
plt.colorbar()
plt.show()

## Remove Annual Cycle

Here we remove the annual cycle, deleting the influence of seasonality from the dataset

In [ ]:
# transition to numpy space
sstx = xr.DataArray.to_numpy(sst1)
mslpx = xr.DataArray.to_numpy(mslp1)

# extract latitude and longitude
latx=len(lat) 
lonx=len(lon)

# extract shape of sstx
sts = np.shape(sstx)
yrmx = int(sts[0]/12)

# Now remove annual cycle from sst and mslp.
#First, reshape:
sstx = np.reshape(sstx,(yrmx,12,latx,lonx))
mslpx = np.reshape(mslpx,(yrmx,12,latx,lonx))

# Now calculate monthly mean
sstm = np.nanmean(sstx,0)
mslpm = np.nanmean(mslpx,0)

if rem_ann:
    # subtract mean from each month
    for i in range(0,yrmx):
        sstx[i,:,:,:] = sstx[i,:,:,:]-sstm
        mslpx[i,:,:,:] = mslpx[i,:,:,:]-mslpm

In [ ]:
np.shape(sstm)
720/12

## Plot a monthly mean - SST

In [ ]:
# let's try to plot a monthly mean

# Plot 1 - January
if True:
    # make figure
    plt.figure(figsize=(12, 4), dpi=100)
    # Set up projection
    ax = plt.axes(projection=ccrs.Robinson(central_longitude=cent_lon))
    # plot data
    plt.contourf(lon, lat, sstm[0,:,:], 60, cmap = col_mapc,
             transform=ccrs.PlateCarree())
    # add gridlines
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                  linewidth=1.5, color='gray', alpha=0.5, linestyle='-')
    # housekeeping
    ax.coastlines()
    plt.colorbar()
    plt.title('Climatological mean SST for January \N{DEGREE SIGN}C')
    plt.show()


# PLOT 2 - July
if True:
    # make figure
    plt.figure(figsize=(12, 4), dpi=100)
    # Set up projection
    ax = plt.axes(projection=ccrs.Robinson(central_longitude=cent_lon))
    # plot data
    plt.contourf(lon, lat, sstm[6,:,:], 60, cmap = col_mapc,
             transform=ccrs.PlateCarree())
    # add gridlines
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                  linewidth=1.5, color='gray', alpha=0.5, linestyle='-')
    # housekeeping
    ax.coastlines()
    plt.colorbar()
    plt.title('Climatological mean SST for July \N{DEGREE SIGN}K')
    plt.show()

## Plot a monthly mean - MSLP

In [ ]:
# let's try to plot a monthly mean

# Plot 1 - January
if True:
    # make figure
    plt.figure(figsize=(12, 4), dpi=100)
    # Set up projection
    ax = plt.axes(projection=ccrs.Robinson(central_longitude=cent_lon))
    # plot data
    plt.contourf(lon, lat, mslpm[0,:,:], 60, cmap = col_mapc,
             transform=ccrs.PlateCarree())
    # add gridlines
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                  linewidth=1.5, color='gray', alpha=0.5, linestyle='-')
    # housekeeping
    ax.coastlines()
    plt.colorbar()
    plt.title('Climatological mean MSLP for January ')
    plt.show()


# PLOT 2 - July
if True:
    # make figure
    plt.figure(figsize=(12, 4), dpi=100)
    # Set up projection
    ax = plt.axes(projection=ccrs.Robinson(central_longitude=cent_lon))
    # plot data
    plt.contourf(lon, lat, mslpm[6,:,:], 60, cmap = col_mapc,
             transform=ccrs.PlateCarree())
    # add gridlines
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                  linewidth=1.5, color='gray', alpha=0.5, linestyle='-')
    # housekeeping
    ax.coastlines()
    plt.colorbar()
    plt.title('Climatological mean MSLP for July')
    plt.show()

## Pick subset of months / reigon for EOF Analysis

In [ ]:
#  Specify the geographical boundaries and months we want to use
# let's experiment a bit with the xarray version
#sst  # this statement tells you what is in the structure sst 
#ssta = sst.sel(time=slice(1950-1-1, 2022-1-1),lon=slice(0., 90.),lat=slice(0., 89.))


# Specify year range, print for user confirmation
yr1= str(yra)+'-01-01'
yr2= str(yrb)+'-12-31'
years = np.linspace(float(yra),float(yrb+1),num = (yrb-yra+1)*12)
print('size years',np.shape(years),yr1,yr2)

# select all data
sst0 = sst.sel(time=slice(yr1, yr2),latitude=slice(90., -90),longitude=slice(0., 360.))
mslp0 = mslp.sel(time=slice(yr1, yr2),latitude=slice(90., -90),longitude=slice(0., 360.))

# select data in specified range
ssta = sst.sel(time=slice(yr1, yr2),latitude=slice(lat1, lat2),longitude=slice(lon1, lon2))
mslpa = mslp.sel(time=slice(yr1, yr2),latitude=slice(lat1, lat2),longitude=slice(lon1, lon2))

# make arrays of lat and lon
lat_sh=ssta.latitude
lon_sh=ssta.longitude

## Remove Global Mean. Weight by sqrt(cosine(latitude)) so area variance is used in EOF computation

In [ ]:
# make vector for weigting the data array for sqrt(cosine(latitude)) so area variance is used in EOF computation
cn = np.sqrt(np.cos(lat_sh*np.pi/180.))

# Apply weighting vector (for use in calculating the global mean)
ssta_weighted = ssta * 0
for i in range(len(lat_sh)):
    # multiply all entries in this row by the weight in the weights vector (cn)
    ssta_weighted[:,i,:] = ssta[:,i,:]*cn[i]  # times sqrt(cos(lat))
print("Weighting Complete")
    
# calculate global mean anomaly
gm = []
for i in range(len(years)):
    gm.append(np.nanmean(ssta_weighted[i,:,:]))

# Apply global mean / area weighting
if rem_globmean:
    
    # first remove global mean
    for i in range(len(years)):
        ssta[i,:,:] = ssta[i,:,:] - gm[i]

    # next apply latitude weighting
    for i in range(len(lat_sh)):
    # multiply all entries in this row by the weight in the weights vector (cn)
        ssta[:,i,:] = ssta[:,i,:]*cn[i]  # times sqrt(cos(lat))

else:
    # if we aren't removing global mean, we still need to apply latitude weighting
    ssta = ssta_weighted
    
# plot global mean anomaly
if True:
    fig,axs = plt.subplots()
    axs.set_ylabel('Global Mean SST Anomaly ($^\circ$C)')
    axs.set_xlabel('Time')
    axs.set_title('Global Mean SST Anomaly (monthly)')
    axs.plot(years,gm)

## Test plot to make sure  all is working


In [ ]:
if True:

    # pick arbitrary month
    i = 150

    # Plot
    plt.figure()
    plt.contourf(lon_sh,lat_sh,ssta[i,:,:]) 
    plt.title('SST anomaly map: '+str(years[i]))
    plt.colorbar()

## Arange Data in a form useful for EOF Analysis and apply land mask

We need an array that is 2-D with all the data points in one direction and all the months in another

Here I double check each point is not land, as the original dataset does a poor job assinging land values

In [ ]:
# set mnx, which defines the number of timesteps
mnx = len(years)

# Convert to numpy
sstx = xr.DataArray.to_numpy(ssta)

# Need to deal with the nan values that represent no land.

# first convert lon to -180 to 180 degrees format
lon_sh_corr=[]
for i in xr.DataArray.to_numpy(lon_sh):
    if i<180:
        lon_sh_corr.append(i)
    else:
        lon_sh_corr.append((360-i)*-1)

# now loop htrough and check each point with global land mask package
icnt = 0
jcnt = 0 
landcnt=0
for i in xr.DataArray.to_numpy(lat_sh):
    jcnt=0
    for j in lon_sh_corr:
        if ~globe.is_ocean(i,j):

            for k in range(mnx):
                sstx[k,icnt,jcnt] = np.nan
            landcnt +=1
        jcnt+=1
    icnt+=1

# return number of conflicts found
print("found ",landcnt,"land conflicts")

# convert into format for xarray
sstx = np.reshape(sstx,(mnx,sstx.shape[1]*sstx.shape[2]))

# start by computing a nan mask
nan_mask = np.isnan(sstx[0,:])
# now remove NaNs
sstx = sstx[:,~nan_mask]

In [ ]:
# Plot to check on the land mask. Should show higher resolution land than the last plot

if True:

    # pick arbitrary month
    i = 150

    # Plot
    plt.figure()
    plt.contourf(lon_sh,lat_sh,ssta[i,:,:]) 
    plt.title('SST anomaly map: '+str(years[i]))
    plt.colorbar()

## Now we can check autocorrelation and estimate decrees of freedom.

In [ ]:
# Ok, now I think we have an array sstz that is [months,space]  Where only land regions are kept
# First let's estimate the number of degrees of freedom in the sample by estimating a grand autocorrelation
def autocorr2(x,lags):
    '''manualy compute, non partial'''

    mean=np.nanmean(x)
    var=np.nanvar(x)
    if var == 0.0:
        var=1.0e-6
    xp=x-mean
    corr=[1. if l==0 else np.sum(xp[l:]*xp[:-l])/len(x)/var for l in lags]
    #corr=[ np.sum(xp[l:]*xp[:-l])/len(x)/var for l in lags]
    return np.array(corr)
xcor = np.zeros([2])

nt, ns = np.shape(sstx)
print('nt',nt,'ns',ns)
type(ns)

lags = [0,1]
for i in range(ns):
    xx=np.squeeze(sstx[:,i])
    xcor1 = autocorr2(xx,lags)
    xcor = xcor + xcor1

print('xx',xx[0:4],xx[-4:])    
xcor = xcor/float(ns)
print('xcor ',xcor)
acorr = xcor[1]
dof_sst = mnx*(1. - acorr**2)/(1. + acorr**2)
print('DOF = ',dof_sst,',   DOF/mnx = ',dof_sst/mnx,',  mnx = ',mnx)

In [ ]:
np.shape(sstx)

## Finally - we can attempt EOF analysis

We're starting this step with sst x, which is lax times lon wide and monthstimes years tall

In [ ]:
# check the shape again
print('sstx shape ',np.shape(sstx))

# do some matrix math!
mmax = np.max(sstx)
print('mmax ',mmax)
u, s, vh = np.linalg.svd(sstx.T,full_matrices=False)
print('u shape',np.shape(u))

## Plot Eigenvalues of EOF / Princial Component Timeseries

In [ ]:
print('u shape',np.shape(u))
print('vh shape',np.shape(vh))
print('s shape',np.shape(s))
type(u)
type(vh)
type(s)
spectrum = s*s.T
spectrum = spectrum/sum(spectrum)
plt.figure(figsize=(12, 4), dpi=100)
yerror = spectrum*np.sqrt(2/dof_sst)
index = np.linspace(0,24,25)
    
plt.errorbar(index[0:12],spectrum[0:12],yerror[0:12],capsize=5)
plt.ylabel('Fraction of Variance')
plt.xlabel('EOF number')
plt.title('Eigenvalue Spectrum - ' + str(lon1) + '-' + str(lon2) + '  ' + str(lat2) +'-' \
              + str(lat1)  )       
#  We need to construct the EOF map by regressing the pc onto the original data
pcmx=4  # were going to consider the first 4 eofs

ts = vh[0:pcmx,:]  # hope this is time series of first eof, YES looks right, has autocorrelation

for pci in range(0,pcmx):

    plt.figure(figsize=(12, 4), dpi=100)
    plt.plot(years,ts[pci,:])
    plt.title('Timeseries of PC-' + str(pci+1) +' - ' + str(lon1) + '-' + str(lon2) + '  ' + str(lat2) +'-' \
              + str(lat1)  )
    plt.grid()


## Regress onto the orignal Data

In [ ]:
# Make empty regression vector
regm = np.empty([pcmx+1,latx*lonx])

# Loop through each PC
for pci in range(0,pcmx):

    # extract the t vector for this PC
    t=ts[pci,:]

    # normalize predictor to have one standard deviation
    t = t/np.std(t)  

    # extract original data, and convert to 2D matrix
    sstb = xr.DataArray.to_numpy(sst0)
    sstb=np.reshape(sstb,(mnx,latx*lonx))

    # compute regression (t*ssb)/mnx
    reg = np.matmul(t,sstb)/mnx

    # save this regression
    regm[pci,:] = reg
    
regm = np.reshape(regm,(pcmx+1,latx,lonx))  # reshape for plotting    

## Plot regressions

In [ ]:
# contour plot some regressions

# set colormaps
col_mapc = 'RdYlBu_r'
col_mapr = 'RdYlBu_r'

# Loop through each PC
for ip in range(0,pcmx):

    # find appropreate regression for this PC
    regm1 = regm[ip,:,:]
    
    # here I am finding the maximum value, then I will fix the contours and colorbar to be constant across months
    if ip == 0:  # same contour interval for all plots
        rmax = np.nanmax(np.abs(regm1))
        nconts=60
        contr = np.linspace(-rmax,rmax,nconts+1)
        print(',  rmax ',rmax, ' - ',np.max(contr))

    # toggle True/False to plot maps of regressions
    if True:
        
        # here we extend the array by one, so there is no gap in the contours
        prcpmp = np.empty([latx,lonx+1])  
        prcpm=regm1
        prcpmp[:,0:len(lon)]= prcpm
        prcpmp[:,len(lon+1)]= prcpm[:,0]

        # make figure
        plt.figure(figsize=(12, 4), dpi=100)

        # set up axes
        ax = plt.axes(projection=ccrs.Robinson(central_longitude=cent_lon))

        # Plot data
        plt.contourf(lonp, lat, prcpmp, contr, cmap=col_mapr,
             transform=ccrs.PlateCarree())

        # add gridlines
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                  linewidth=1.5, color='gray', alpha=0.5, linestyle='-')

        # housekeeping
        ax.coastlines()
        plt.colorbar()
        plt.title('Regression for EOF ' + str(ip+1) + ' \N{DEGREE SIGN}C ' + str(yra) + '-' + str(yrb) + '  ' + str(lon1) + '-' + str(lon2) + '  ' + str(lat2) +'-' \
              + str(lat1) )
        plt.show()


In [ ]:
np.shape(sstx)

In [ ]:
1488/12